In [5]:
from model.grid import generateGrid
from model.propabilities import calculateTransitionPropabilitiesForAllPorfolios, select_probabilities_for_chosen_strategies, calculate_cumulative_propabilities
from model.investment_planner import get_portfolios_strategies, get_optimal_strategies_for_T
from model._utilities import Goals
import numpy as np
import pandas as pd
import seaborn as sns

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
portfolios = np.array([[0.0526, 0.037 ],
       [0.0552, 0.0396],
       [0.0577, 0.0463],
       [0.0603, 0.0556],
       [0.0629, 0.0664],
       [0.0655, 0.0781],
       [0.068 , 0.0904],
       [0.0706, 0.1031],
       [0.0732, 0.1159],
       [0.0757, 0.129 ],
       [0.0783, 0.1421],
       [0.0809, 0.1554],
       [0.0834, 0.1687],
       [0.086 , 0.1821],
       [0.0886, 0.1955]])

In [26]:
W0 = 100
iMax = 475

goals = Goals([{
        "time": 5,
        "cost": 100,
        "utility": 1000                
    },
    {
        "time": 10,
        "cost": 150,
        "utility": 1000                
    },  
    ])
T = goals.get_investment_period()
goal_max_costs = goals.get_highest_costs()
infusions = np.full(T,0)


In [27]:
grid = generateGrid(W0,iMax,infusions,goal_max_costs,portfolios[0,0],portfolios[0,1],portfolios[-1,0],portfolios[-1,1])

goals_strategies = np.zeros((T,iMax))
portfolios_strategies = np.zeros((T,iMax))
probabilities = np.zeros((T,iMax,iMax))
V = np.zeros((T,iMax))
selected_propabilities = np.zeros((T-1,iMax,iMax))

for t in range(T-2,-1,-1):
    probabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,grid[t],grid[t+1],infusions[t],1)
    result = get_optimal_strategies_for_T(goals.get_k_array(t),grid[t], probabilities, V[t+1])  
    goals_strategies[t] = result.goals_strategies
    portfolios_strategies[t] = result.porfolios_strategies
    V[t] = result.values
    goals_costs = goals.get_costs_for_time(t)
    selected_propabilities[t] = select_probabilities_for_chosen_strategies(probabilities,result.porfolios_strategies,result.goals_strategies,goals_costs,grid[t])
    

In [28]:
W0index = np.where(grid[0]==W0)[0][0]
calculate_cumulative_propabilities(selected_propabilities,goals_strategies[1:,:],W0index)

{1: {0.0: 1.0},
 2: {0.0: 1.0},
 3: {0.0: 1.0},
 4: {0.0: 0.198, 1.0: 0.802},
 5: {0.0: 1.0},
 6: {0.0: 1.0},
 7: {0.0: 1.0},
 8: {0.0: 1.0},
 9: {0.0: 0.941, 1.0: 0.059}}

In [30]:
df = pd.DataFrame(portfolios_strategies.T)
df.insert(0,'W',grid[1].T)
#df.set_index('W')
#sns.heatmap(df.iloc[230:251])
short = df.iloc[244:324]


TODO:
* test get_portfolios_strategies when vt1 for all i
* dodać wyświetlanie portfolios strategies i goal strategies a grid wealth values (data frame)
* wyświetlić z multiple goals